In [2]:
import pandas as pd
import numpy as np
import math

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#from geopy.distance import geodesic

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

#Skip to reading Step 2. way down 

ModuleNotFoundError: No module named 'geopy'

In [3]:
dataset = pd.read_csv("merged_training_dataset.csv")

In [4]:
dataset = dataset[dataset['order_status'] == 'delivered']
len(dataset)

115728

In [5]:
dataset.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'quantity', 'product_id', 'seller_id', 'shipping_limit_date',
       'product_price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category', 'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'seller_lat', 'sellet_lng', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'customer_lat', 'customer_lng'],
      dtype='object')

In [22]:
dataset.dropna(subset=[ 'order_delivered_customer_date', 'order_delivered_carrier_date', 'order_approved_at', 'seller_lat', 'customer_lat', 'product_category'],inplace=True)
len(dataset)

113487

In [23]:
dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,quantity,product_id,...,seller_city,seller_state,seller_lat,sellet_lng,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13T08:59:02Z,2017-09-13T09:45:35Z,2017-09-19T18:34:16Z,2017-09-20T23:43:48Z,2017-09-29T00:00:00Z,1,4244733e06e7ecb4970a6e2683c13e61,...,volta redonda,SP,-22.498183,-44.123614,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,-21.758076,-41.312633
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26T10:53:06Z,2017-04-26T11:05:13Z,2017-05-04T14:35:00Z,2017-05-12T16:04:24Z,2017-05-15T00:00:00Z,1,e5f2d52b802189ee658865ca93d83a8f,...,sao paulo,SP,-23.566258,-46.518417,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,-20.212393,-50.941471
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14T14:33:31Z,2018-01-14T14:48:30Z,2018-01-16T12:36:48Z,2018-01-22T13:19:16Z,2018-02-05T00:00:00Z,1,c777355d18b72b67abbeef9df44fd0fd,...,borda da mata,MG,-22.264094,-46.158564,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,-19.860439,-44.597972
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08T10:00:35Z,2018-08-08T10:10:18Z,2018-08-10T13:28:00Z,2018-08-14T13:32:39Z,2018-08-20T00:00:00Z,1,7634da152a4610f1595efa32f14722fc,...,franca,SP,-20.548228,-47.395897,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,-23.144923,-46.539830
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04T13:57:51Z,2017-02-04T14:10:13Z,2017-02-16T09:46:09Z,2017-03-01T16:42:31Z,2017-03-17T00:00:00Z,1,ac6c3623068f30de03045865e4e10089,...,loanda,PR,-22.931427,-53.133759,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,-23.249008,-46.824961


In [24]:
dataset.order_purchase_timestamp = pd.to_datetime(dataset.order_purchase_timestamp)
#dataset.order_aproved_at = pd.to_datetime(dataset.order_aproved_at)
dataset.order_estimated_delivery_date = pd.to_datetime(dataset.order_estimated_delivery_date)
dataset.order_delivered_customer_date = pd.to_datetime(dataset.order_delivered_customer_date)
dataset.order_approved_at = pd.to_datetime(dataset.order_approved_at)
dataset.shipping_limit_date = pd.to_datetime(dataset.shipping_limit_date)
dataset.order_delivered_carrier_date = pd.to_datetime(dataset.order_delivered_carrier_date)
dataset.review_creation_date = pd.to_datetime(dataset.review_creation_date)
dataset.review_answer_timestamp = pd.to_datetime(dataset.review_answer_timestamp)
dataset.product_category.replace(['None', 'NaN', np.nan], "NIL", inplace=True)
dataset['freight_rate'] = dataset[['freight_value','quantity']].apply(
    lambda row : round(row['freight_value'] / row['quantity'],2), axis=1
)

#dataset.pivot(index='', columns='name', values='dollars')

In [25]:
columns = ['order_id', 'product_id', 'seller_id', 'customer_id', 'customer_unique_id','review_id', 'quantity', 'payment_type']
#dataset = dataset.drop("index", axis = 1)
dataset['payment_sum'] = dataset.groupby(['order_id', 'product_id', 'seller_id', 'customer_id', 'customer_unique_id','review_id', 'quantity', 'payment_type'])['payment_value'].transform('sum')
dataset.drop_duplicates(subset = columns, keep = "first", inplace = True)

In [26]:
dataset2 = pd.pivot_table(dataset, index=['order_id', 'product_id', 'seller_id', 'customer_id', 'customer_unique_id','review_id', 'quantity']
               , columns='payment_type', values='payment_sum', fill_value = 0).reset_index()

In [27]:
columns.remove("payment_type")
dataset.drop_duplicates(subset = columns, keep = "first", inplace = True)

In [28]:
dataset = pd.merge(dataset,dataset2, on = ['order_id', 'product_id', 'seller_id', 'customer_id', 'customer_unique_id','review_id', 'quantity'], how = "right")

In [29]:
dataset.drop(['payment_type', 'payment_sum'], axis = 1, inplace = True)

In [30]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108716 entries, 0 to 108715
Data columns (total 47 columns):
order_id                         108716 non-null object
customer_id                      108716 non-null object
order_status                     108716 non-null object
order_purchase_timestamp         108716 non-null datetime64[ns, UTC]
order_approved_at                108716 non-null datetime64[ns, UTC]
order_delivered_carrier_date     108716 non-null datetime64[ns, UTC]
order_delivered_customer_date    108716 non-null datetime64[ns, UTC]
order_estimated_delivery_date    108716 non-null datetime64[ns, UTC]
quantity                         108716 non-null int64
product_id                       108716 non-null object
seller_id                        108716 non-null object
shipping_limit_date              108716 non-null datetime64[ns, UTC]
product_price                    108716 non-null float64
freight_value                    108716 non-null float64
payment_sequential        

In [35]:
def split_train_test(data, test_ratio):
    np.random.seed(1234)    
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [36]:
train, test = split_train_test(dataset, 0.8)

In [37]:
y_train = train['review_score'] >= 3 
y_test = test['review_score'] >= 3

In [38]:
def performance_metrics(y, y_pred):
    accuracy = sum(y_pred == y)/len(y)
    c_m = confusion_matrix(y, y_pred)
    TP = c_m[1][1]
    FP = c_m[0][1]
    FN = c_m[1][0]
    TN = c_m[0][0]
    print(confusion_matrix(y, y_pred))
    BER = 1 - 0.5 * (TP / (TP + FN) + TN / (TN + FP))
    return accuracy, BER

In [39]:
categories = dataset['product_category'].unique().tolist()
seller_popularities = pd.DataFrame(train.groupby('seller_id').size().reset_index(name = "Count"))

def features(data):
    data.fillna(0)
    
    feat1 = [data['product_category'] == c for c in categories]
    review_length = 0 if (pd.isnull(data['review_comment_message'])) else len(data['review_comment_message'])
    delivery_time = data['order_delivered_customer_date'] - data['order_estimated_delivery_date']
    approving_time = data['order_approved_at'] - data['order_purchase_timestamp']
    processing_time = data['order_delivered_carrier_date'] - data['order_approved_at'] 
    shipping_time = data['shipping_limit_date'] - data['order_delivered_carrier_date']
    purchase_month = data['order_purchase_timestamp'].month
    credit_pay = data['credit_card']
    debit_pay = data['debit_card']
    voucher_pay = data['voucher']
    boleto_wallet_pay = data['boleto']
    price = 0 if (pd.isnull(data['product_price'])) else data['product_price']
    desc_length = 0 if (pd.isnull(data['product_description_lenght'])) else data['product_description_lenght']
    photos = 0 if (pd.isnull(data['product_photos_qty'])) else data['product_photos_qty']
    price = data['product_price']
    time_to_review = data['review_answer_timestamp'] - data['order_delivered_customer_date']
    seller_coord = (data['seller_lat'], data['sellet_lng'])
    customer_coord = (data['customer_lat'], data['customer_lng'])
    distance = geodesic(seller_coord, customer_coord).miles
    popularity = seller_popularities[seller_popularities["seller_id"] == data["seller_id"]].Count.values[0] if data["seller_id"] in seller_popularities.seller_id else 0
    freight_rate = data["freight_rate"]
    
    return feat1 + [review_length, delivery_time.days, approving_time.days, approving_time.days, processing_time.days, 
                    shipping_time.days, purchase_month, credit_pay, debit_pay, voucher_pay, boleto_wallet_pay, desc_length,
                    photos, price, distance, time_to_review.days, popularity, freight_rate]

features(dataset.iloc[1,])

[False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 0,
 -3,
 0,
 0,
 8,
 -2,
 4,
 259.83,
 0.0,
 0.0,
 0.0,
 239.0,
 2.0,
 239.9,
 365.8932474414344,
 2,
 0,
 19.93]

In [40]:
X_train = [features(row) for _, row in train.iterrows()]
X_test = [features(row) for _, row in test.iterrows()]

In [42]:
min_max_scaler = preprocessing.MinMaxScaler()
std_X_train = min_max_scaler.fit_transform(X_train)
std_X_test = min_max_scaler.transform(X_test)

In [88]:
log_reg = LogisticRegression(C = 1.0)
random_forest = RandomForestClassifier(n_estimators=1000, max_features = 2)
lin_SVM = SVC(kernel="linear", C=1.0)
knn_classifier = KNeighborsClassifier(n_neighbors=5)

In [74]:
accuracy = cross_val_score(log_reg, std_X_train, y_train, scoring='accuracy', cv = 5).mean() * 100
print("Accuracy of Logistic Regresssion is: " , accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Logistic Regresssion is:  86.4605751016482


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
accuracy = cross_val_score(random_forest, std_X_train, y_train, scoring='accuracy', cv = 5).mean() * 100
print("Accuracy of Random Forest is: " , accuracy) 

In [87]:
grid_values = {'n_estimators': [1, 2, 3, 5, 8, 11, 19, 30, 49, 79, 128, 207, 250, 500, 1000], 'max_features': [2, 4, 6, 8]}

grid_search = GridSearchCV(random_forest, param_grid = grid_values,scoring = 'recall', cv = 5, return_train_score=True)

grid_search.fit(std_X_train, y_train)

grid_search.best_params_

{'max_features': 2, 'n_estimators': 1000}

In [ ]:
accuracy = cross_val_score(lin_SVM, std_X_train, y_train, scoring='accuracy', cv = 5).mean() * 100
print("Accuracy of Random Forest is: " , accuracy)

In [75]:
accuracy = cross_val_score(knn_classifier, std_X_train, y_train, scoring='accuracy', cv = 5).mean() * 100
print("Accuracy of KNN is: ", accuracy)

Accuracy of Random Forest is:  85.75233774385278


In [52]:
log_reg.fit(std_X_train, y_train)
log_reg.score(std_X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8630938692912662

In [89]:
random_forest.fit(std_X_train, y_train)
random_forest.score(std_X_test, y_test)

0.8912753529871683

In [66]:
lin_SVM.fit(std_X_train, y_train)
lin_SVM.score(std_X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8629099020374373

In [76]:
knn_classifier.fit(std_X_train, y_train)
knn_classifier.score(std_X_test, y_test)

0.8523777767557374

In [55]:
log_reg = LogisticRegression(C = 1.0)
log_reg.fit(std_X_train, y_train)
y_train_pred = log_reg.predict(std_X_train)
performance_metrics(y_train, y_train_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[[  853  2365]
 [  546 17980]]


(0.8661239882266373, 0.38220031015487144)

In [56]:
y_test_pred = log_reg.predict(std_X_test)
performance_metrics(y_test, y_test_pred)

[[ 3535  9702]
 [ 2205 71530]]


(0.8630938692912662, 0.3814249593989252)

In [57]:
random_forest = RandomForestClassifier(n_estimators=5)
random_forest.fit(std_X_train,y_train)
y_train_pred = random_forest.predict(std_X_train)
performance_metrics(y_train, y_train_pred)

[[ 2921   297]
 [   85 18441]]


(0.9824319352465047, 0.048440748147816626)

In [58]:
y_test_pred = random_forest.predict(std_X_test)
performance_metrics(y_test, y_test_pred)

[[ 5475  7762]
 [ 2879 70856]]


(0.8776502782504714, 0.31271593651874674)

In [54]:
performance_metrics(y_test, [1] * len(y_test))

[[    0 13237]
 [    0 73735]]


(0.8478015913167456, 0.5)